In [1]:
import torch
import torchphysics as tp
X = tp.spaces.R1('x')
Y = tp.spaces.R1('y')
Z = tp.spaces.R1('z')
T = tp.spaces.R1('t')
U = tp.spaces.R1('u')
V = tp.spaces.R1('v')
W = tp.spaces.R1('w')

URMS=tp.spaces.R1("urms")
WRMS=tp.spaces.R1("vrms")
VRMS=tp.spaces.R1("wrms")
P=tp.spaces.R1('p')

/net/istmhome/users/hi224/Dokumente/Python/TorchPhysics/TP/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np

In [3]:
class Turbulent_Data(torch.utils.data.Dataset):
    def __init__(self,data_DF,x_list,y_list):
        self.x_train=data_DF[x_list].values
        self.y_train=data_DF[y_list].values
        #self.x_train=torch.tensor(x_train,dtype=torch.float32)
        #self.y_train=torch.tensor(y_train,dtype=torch.float32)
        #self.x_train=tp.spaces.Points(self.x_train,X*Y)
        #self.y_train=tp.spaces.Points(self.y_train,U*V)
    def __len__(self):
        return len(self.y_train)

    def __getitem__(self,idx):
        #return self.x_train[idx] , self.y_train[idx]
        #print(torch.as_tensor([self.x_train[idx]]).shape)
        P_xtrain=tp.spaces.Points([self.x_train[idx]],X*Y,dtype=torch.float32)
        P_ytrain=tp.spaces.Points([self.y_train[idx]],U*V*URMS*VRMS*UV,dtype=torch.float32)
        return P_xtrain,P_ytrain

In [4]:
class Ho_Turbulent_Data_mean(torch.utils.data.Dataset):
    def __init__(self,data_DF,x_list,y_list,horizon_sampler):
        self.x_train=data_DF[x_list].values
        self.y_train=data_DF[y_list].values
        self.sampler=horizon_sampler
        #self.x_train=torch.tensor(x_train,dtype=torch.float32)
        #self.y_train=torch.tensor(y_train,dtype=torch.float32)
        #self.x_train=tp.spaces.Points(self.x_train,X*Y)
        #self.y_train=tp.spaces.Points(self.y_train,U*V)
    def __len__(self):
        return len(self.y_train)

    def __getitem__(self,idx):
        #return self.x_train[idx] , self.y_train[idx]
        #print(torch.as_tensor([self.x_train[idx]]).shape)
        x_horizon= self.sampler.sample_points(device="cpu")
        x_horizon,_=x_horizon.track_coord_gradients()
        #print(x_horizon)
        x_horizon_x=x_horizon["x"].detach().numpy()
        x_horizon_z=x_horizon["z"].detach().numpy()
        x_horizon_t=x_horizon["t"].detach().numpy()
        y_new=np.broadcast_to(self.x_train[idx],(x_horizon_x.shape[0],self.x_train[idx].shape[0]))
        target_new=np.broadcast_to(self.y_train[idx],(x_horizon_x.shape[0],self.y_train[idx].shape[0]))
        x_horizon_x = np.concatenate((x_horizon_x,y_new),axis=1)
        x_horizon_x=np.concatenate((x_horizon_x, x_horizon_z), axis=-1)
        X_points=np.concatenate((x_horizon_x, x_horizon_t), axis=-1)
        P_xtrain=tp.spaces.Points([X_points],X*Y*Z*T,dtype=torch.float32)
        P_ytrain=tp.spaces.Points([target_new],U*V*W,dtype=torch.float32)
        return P_xtrain,P_ytrain


class Ho_Turbulent_Data_rms(torch.utils.data.Dataset):
    def __init__(self,data_DF,x_list,y_list,horizon_sampler):
        self.x_train=data_DF[x_list].values
        self.y_train=data_DF[y_list].values
        self.sampler=horizon_sampler
        #self.x_train=torch.tensor(x_train,dtype=torch.float32)
        #self.y_train=torch.tensor(y_train,dtype=torch.float32)
        #self.x_train=tp.spaces.Points(self.x_train,X*Y)
        #self.y_train=tp.spaces.Points(self.y_train,U*V)
    def __len__(self):
        return len(self.y_train)

    def __getitem__(self,idx):
        #return self.x_train[idx] , self.y_train[idx]
        #print(torch.as_tensor([self.x_train[idx]]).shape)
        x_horizon= self.sampler.sample_points(device="cpu")
        x_horizon,_=x_horizon.track_coord_gradients()
        x_horizon_x=x_horizon["x"].detach().numpy()
        x_horizon_z=x_horizon["z"].detach().numpy()
        x_horizon_t=x_horizon["t"].detach().numpy()
        y_new=np.broadcast_to(self.x_train[idx],(x_horizon_x.shape[0],self.x_train[idx].shape[0]))
        target_new=np.broadcast_to(self.y_train[idx],(x_horizon_x.shape[0],self.y_train[idx].shape[0]))
        x_horizon_x = np.concatenate((x_horizon_x,y_new),axis=1)
        x_horizon_x=np.concatenate((x_horizon_x, x_horizon_z), axis=-1)
        X_points=np.concatenate((x_horizon_x, x_horizon_t), axis=-1)
        P_xtrain=tp.spaces.Points([X_points],X*Y*Z*T,dtype=torch.float32)
        P_ytrain=tp.spaces.Points([target_new],URMS*VRMS*WRMS,dtype=torch.float32)
        return P_xtrain,P_ytrain

In [5]:
class Rn_Turbulent_Data_mean(torch.utils.data.Dataset):
    def __init__(self,data_DF,x_list,y_list,max_it):
        self.x_train=data_DF[x_list].values
        self.y_train=data_DF[y_list].values
        self.length=min(max_it,len(self.y_train))
        #self.x_train=torch.tensor(x_train,dtype=torch.float32)
        #self.y_train=torch.tensor(y_train,dtype=torch.float32)
        #self.x_train=tp.spaces.Points(self.x_train,X*Y)
        #self.y_train=tp.spaces.Points(self.y_train,U*V)
    def __len__(self):
        return self.length

    def __getitem__(self,idx):
        #return self.x_train[idx] , self.y_train[idx]
        #print(torch.as_tensor([self.x_train[idx]]).shape)
        id_random=random.randint(0,len(self.y_train))
        P_xtrain=tp.spaces.Points([self.x_train[id_random]],X*Y,dtype=torch.float32)
        P_ytrain=tp.spaces.Points([self.y_train[id_random]],U*V*URMS*VRMS*UV,dtype=torch.float32)
        return P_xtrain,P_ytrain

In [6]:
DF_Data=pd.read_csv("Data/Flat_Turb_500.csv")

In [7]:
DF=DF_Data#[0::5]

In [8]:
#DF=DF[(DF["y"]<=0.2)|(DF["y"]>=1.8)]

In [9]:
DF

,x,y,U,V,W,urms,vrms,wrms,uv,uw,vw
0,0,0.000000,-7.184000e-16,-2.267000e-35,1.309300e-19,1.967500e-14,7.751400e-17,1.005400e-15,4.814100e-35,-1.826400e-32,4.935100e-36
1,0,0.000031,1.286900e-03,3.593100e-26,-5.444100e-08,5.187100e-04,2.314900e-07,3.316000e-04,-2.474000e-11,-5.954200e-11,-6.048400e-15
2,0,0.000123,5.147100e-03,6.249800e-25,-2.177600e-07,2.074000e-03,3.681500e-06,1.319400e-03,-1.590600e-09,-9.656000e-10,-4.078300e-13
3,0,0.000278,1.158000e-02,-1.264700e-24,-4.899200e-07,4.662800e-03,1.842900e-05,2.940800e-03,-1.823100e-08,-4.999500e-09,-4.944100e-12
4,0,0.000493,2.058300e-02,-3.911600e-24,-8.708500e-07,8.281000e-03,5.731000e-05,5.159400e-03,-1.033400e-07,-1.626900e-08,-3.008100e-11
...,...,...,...,...,...,...,...,...,...,...,...
396,0,1.999500,2.049700e-02,7.117400e-24,-1.198600e-05,8.230900e-03,5.700000e-05,5.128000e-03,1.027000e-07,-2.976500e-07,-8.451900e-11
397,0,1.999700,1.153100e-02,-4.540300e-25,-6.742500e-06,4.634600e-03,1.832800e-05,2.922900e-03,1.812100e-08,-9.426500e-08,-1.498500e-11
398,0,1.999900,5.125500e-03,2.467800e-25,-2.996700e-06,2.061400e-03,3.661300e-06,1.311400e-03,1.581000e-09,-1.863300e-08,-1.311500e-12
399,0,2.000000,1.281500e-03,4.035400e-26,-7.491900e-07,5.155700e-04,2.302200e-07,3.295800e-04,2.459300e-11,-1.165100e-09,-2.043400e-14


In [10]:
import random
def Augment_1D_data(DF,Nsampler,L_x):
    DF_out=DF[0:0]
    for _ in range(Nsampler):
        Ny=random.randint(0, len(DF)-1)
        DF_temp=DF.iloc[[Ny]]
        DF_temp["x"]=random.uniform(0,L_x)
        DF_out=pd.concat([DF_out,DF_temp],ignore_index=True)
    return DF_out

        

In [11]:
torch.cuda.device_count()

0

In [12]:
X_interval = tp.domains.Interval(X, 0, 4.0) # <-add the bounds of the Interval (0, 2)
Y_interval = tp.domains.Interval(Y, 0, 2.0)
Z_interval = tp.domains.Interval(Z, 0, 2.0)
T_interval = tp.domains.Interval(T, 0, 100 )
Sim_domain = X_interval*Y_interval*Z_interval*T_interval


In [13]:
DF_out=Augment_1D_data(DF_Data,8000,2.0)
ho_sampler = tp.samplers.RandomUniformSampler(X_interval*Z_interval*T_interval, n_points=1000)
dataset_turbulent_Mean=Ho_Turbulent_Data_mean(DF,["y"],["U","V","W"],ho_sampler)
dataset_turbulent_Rms=Ho_Turbulent_Data_rms(DF,["y"],["urms","vrms","wrms"],ho_sampler)

/tmp/ipykernel_2426853/1272658583.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_temp["x"]=random.uniform(0,L_x)


In [14]:
bound_sampler = tp.samplers.RandomUniformSampler(X_interval*Y_interval.boundary*Z_interval*T_interval, n_points=500)


In [15]:
def IBM_filter(y):
    return y[...,0]<0
    
IBM_sampler = tp.samplers.RandomUniformSampler(Sim_domain,n_points=1000,filter_fn=IBM_filter)

In [16]:
def IBM_filter_2(y):
    return (y[...,0]<0)&(y[...,0]>-0.1)
    
IBM_sampler_2 = tp.samplers.RandomUniformSampler(Sim_domain,n_points=1000,filter_fn=IBM_filter_2)

In [17]:
def Inner_filter(y):
    return y[...,0]>=0

inner_sampler = tp.samplers.RandomUniformSampler(Sim_domain, n_points=3000)#,filter_fn=Inner_filter)

In [18]:
model = tp.models.FCN(input_space=X*Y*Z*T, output_space=U*V*W*P, hidden=(64,64,64,64,64,64))#######Activation????

In [19]:
data_cond_mean = tp.conditions.DataCondition_mean(model, dataset_turbulent_Mean, norm=2.0,root=1.0,weight=50,use_full_dataset=True,name='Data_Mean')
data_cond_rms = tp.conditions.DataCondition_rms(model, dataset_turbulent_Rms, norm=2.0,root=1.0,weight=5000,use_full_dataset=True,name='Data_RMS')

In [20]:
def pde_IBM(u,v,w,urms,vrms,wrms,uv,uw,vw):
    return torch.abs(u)+torch.abs(v)+torch.abs(w)
pde_cond_IBM = tp.conditions.PINNCondition(model, IBM_sampler, pde_IBM,weight=10,name='IBM')

In [21]:
def pde_IBM_2(u,v):
    return torch.abs(u)+torch.abs(v)+torch.abs(w)
pde_cond_IBM_2 = tp.conditions.PINNCondition(model, IBM_sampler_2, pde_IBM_2,weight=10,name='IBM_2')

In [22]:
def pde_mass(u,v,x,y):
    return tp.utils.grad(u,x)+tp.utils.grad(v,y)-0.0
pde_cond_mass = tp.conditions.PINNCondition(model, inner_sampler,pde_mass, weight=600,name='Conti')

In [23]:
def pde_residual_x(u,v,w, x, y,z,p,t):
    residual_momentum= tp.utils.grad(u,t)+u*tp.utils.grad(u,x)+ v*tp.utils.grad(u,y)+w*tp.utils.grad(u,z)+(-tp.utils.laplacian(u, x)-tp.utils.laplacian(u,y)-tp.utils.laplacian(u,z))/6000 + (1/12)**2+tp.utils.grad(p,x)
    return residual_momentum

pde_cond_x = tp.conditions.PINNCondition(model, inner_sampler, pde_residual_x, pde_IBM,weight=600,name='Momentum_x')

In [24]:
def pde_residual_y(u,v,w, x, y,z,p,t):
    residual_momentum= tp.utils.grad(v,t)+u*tp.utils.grad(v,x)+ v*tp.utils.grad(v,y)+w*tp.utils.grad(v,z)+(-tp.utils.laplacian(v, x)-tp.utils.laplacian(v,y)-tp.utils.laplacian(v,z))/6000+tp.utils.grad(p,y)
    return residual_momentum

pde_cond_y = tp.conditions.PINNCondition(model, inner_sampler, pde_residual_y, pde_IBM,weight=600,name='Momentum_y')

In [25]:
def pde_residual_z(u,v,w, x, y,z,p,t):
    residual_momentum= tp.utils.grad(w,t)+u*tp.utils.grad(w,x)+ v*tp.utils.grad(w,y)+w*tp.utils.grad(w,z)+(-tp.utils.laplacian(w, x)-tp.utils.laplacian(w,y)-tp.utils.laplacian(w,z))/6000+tp.utils.grad(p,z)
    return residual_momentum

pde_cond_z = tp.conditions.PINNCondition(model, inner_sampler, pde_residual_y, pde_IBM,weight=600,name='Momentum_z')

In [26]:
def boundary_residual_x(u, x,y):
    return torch.abs(u) - 0.0

boundary_cond_x = tp.conditions.PINNCondition(model, bound_sampler, boundary_residual_x, weight=100,name='noslip_x')

In [27]:
def boundary_residual_y(v, x,y):
    return torch.abs(v) - 0.0

boundary_cond_y = tp.conditions.PINNCondition(model, bound_sampler, boundary_residual_y, weight=100,name='noslip_y')

In [28]:
def boundary_residual_z(w, x,y):
    return torch.abs(w) - 0.0

boundary_cond_z = tp.conditions.PINNCondition(model, bound_sampler, boundary_residual_x, weight=100,name='noslip_z')

In [29]:
Periodic_sampler=tp.samplers.RandomUniformSampler(Y_interval*Z_interval*T_interval,n_points=250)#,filter_fn=Inner_filter)
Periodic_sampler_Per=tp.samplers.RandomUniformSampler(X_interval*Y_interval*T_interval,n_points=250)#,filter_fn=Inner_filter)

In [30]:
def periodic_residual_x(u_left,u_right):
    Periodic_condition= u_left - u_right
    return Periodic_condition
periodic_cond_x=tp.conditions.PeriodicCondition(model,X_interval,periodic_residual_x,non_periodic_sampler=Periodic_sampler, weight=1,name='periodic_x')
periodic_cond_x_Per=tp.conditions.PeriodicCondition(model,Z_interval,periodic_residual_x,non_periodic_sampler=Periodic_sampler_Per, weight=1,name='periodic_x_Per')

In [31]:
def periodic_residual_y(v_left,v_right):
    Periodic_condition= v_left - v_right
    return Periodic_condition
periodic_cond_y=tp.conditions.PeriodicCondition(model,X_interval,periodic_residual_y,non_periodic_sampler=Periodic_sampler, weight=1,name='periodic_y')
periodic_cond_y_Per=tp.conditions.PeriodicCondition(model,Z_interval,periodic_residual_y,non_periodic_sampler=Periodic_sampler_Per, weight=1,name='periodic_y_Per')

In [32]:
def periodic_residual_z(w_left,w_right):
    Periodic_condition= w_left - w_right
    return Periodic_condition
periodic_cond_z=tp.conditions.PeriodicCondition(model,X_interval,periodic_residual_z,non_periodic_sampler=Periodic_sampler, weight=1,name='periodic_z')
periodic_cond_z_Per=tp.conditions.PeriodicCondition(model,Z_interval,periodic_residual_z,non_periodic_sampler=Periodic_sampler_Per, weight=1,name='periodic_z_Per')

In [33]:
def periodic_residual_p(p_left,p_right):
    Periodic_condition= p_left-p_right
    return Periodic_condition
periodic_cond_p=tp.conditions.PeriodicCondition(model,X_interval,periodic_residual_p,non_periodic_sampler=Periodic_sampler, weight=1,name='periodic_p')
periodic_cond_p_Per=tp.conditions.PeriodicCondition(model,Z_interval,periodic_residual_p,non_periodic_sampler=Periodic_sampler_Per, weight=1,name='periodic_p_Per')

In [34]:
bound_sampler_left = tp.samplers.RandomUniformSampler(X_interval.boundary_left*Y_interval*Z_interval*T_interval, n_points=250)
bound_sampler_left_Per = tp.samplers.RandomUniformSampler(X_interval*Y_interval*Z_interval.boundary_left*T_interval, n_points=250)

def boundary_residual_p(p, x,y):
    return p

boundary_cond_p = tp.conditions.PINNCondition(model, bound_sampler_left, boundary_residual_p, weight=1,name='bound_p')
boundary_cond_p_Per = tp.conditions.PINNCondition(model, bound_sampler_left_Per, boundary_residual_p, weight=1,name='bound_p_per')

In [35]:
##Learning rate scheduling To-Do -- launch LR scheduling only after first training phase
optim = tp.OptimizerSetting(torch.optim.Adam, lr=0.001,scheduler_class=torch.optim.lr_scheduler.ReduceLROnPlateau,scheduler_args={"patience":1000,"factor":0.8,"verbose":True,"min_lr":0.000005},monitor_lr="train/loss")
#solver = tp.solver.Solver([pde_cond_IBM,pde_cond_mass,boundary_cond_x, pde_cond_x,periodic_cond_x,boundary_cond_y, pde_cond_y,periodic_cond_y], optimizer_setting=optim)
##loss terms scheduling
solver = tp.solver.Solver([data_cond_mean,
                           data_cond_rms,
                           boundary_cond_x,#1000
                           boundary_cond_y,
                           boundary_cond_z,#1000
                           boundary_cond_p,
                           boundary_cond_p_Per,
                           periodic_cond_x,#2000
                           periodic_cond_x_Per,#2000
                           periodic_cond_y,
                           periodic_cond_y_Per,
                           periodic_cond_z,#2000
                           periodic_cond_z_Per,
                           periodic_cond_p,
                           periodic_cond_p_Per,
                           pde_cond_x,#5000
                           pde_cond_y,#5000
                           pde_cond_z,
                           pde_cond_mass],#1000
                          optimizer_setting=optim,
                         loss_function_schedule=[{
                        "conditions":list(range(19)),
                        "max_iter":50000
                    }
                ],
                          weight_tunning=True,
                          weight_tunning_parameters={
                                    "alfa":0.99,
                                    "E_rho":0.99,
                                    "Temperature":0.1,
                                    "tunning_every_n_steps":100
                          } ## Default weight-tunning settings
                         )

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
torch.set_float32_matmul_precision('medium')
comet_logger = pl_loggers.CSVLogger(save_dir="logs/")
trainer = pl.Trainer(gpus=0,# use one GPU
                     max_steps=20000, # iteration number
                     benchmark=True, # faster if input batch has constant size
                     logger=comet_logger, # for writting into tensorboard
                     log_every_n_steps=100,
                     enable_checkpointing=False) # saving checkpoints ToDo: turn on checkpointing after first training phase
trainer.fit(solver)

/net/istmhome/users/hi224/Dokumente/Python/TorchPhysics/TP/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/net/istmhome/users/hi224/Dokumente/Python/TorchPhysics/TP/lib/python3.8/site-packages/pytorch_lightning/core/optimizer.py:273: RuntimeWarning: Found unsupported keys in the lr scheduler dict: {'factor', 'verbose', 'min_lr', 'patience'}. HINT: remove them from the output of `configure_optimizers`.
  rank_zero_warn(

  | Name             | Type       | Params
------------------------------------------------
0 | train_conditions | ModuleList | 21.4 K
1 | val_conditions   | ModuleList | 0     
------------

/net/istmhome/users/hi224/Dokumente/Python/TorchPhysics/TP/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/net/istmhome/users/hi224/Dokumente/Python/TorchPhysics/TP/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|                                                            | 0/20001 [00:00<?, ?it/s]

/net/istmhome/users/hi224/Dokumente/Python/TorchPhysics/TP/lib/python3.8/site-packages/torchphysics/problem/spaces/points.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  self._t = torch.as_tensor(data, **kwargs)


Epoch 0:   5%|█▏                      | 977/20001 [1:11:12<23:06:31,  4.37s/it, loss=38.4, v_num=89]

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.close()

In [ ]:
def produce_sample(x_max,y_max,z_position,t_max,resolution=100,frames_t=50):
    list_x=np.linspace(0,x_max,resolution)
    list_y=np.linspace(0,y_max,resolution)
    #list_z=np.ones(resolution)*z_position
    list_t=np.linspace(0,t_max,frames_t)
    #XM,_,_=np.meshgrid(list_x,list_y,list_t)
    XM2D,YM2D=np.meshgrid(list_x,list_y)
    model_value_u=np.zeros(XM2D.shape)
    model_value_v=np.zeros(XM2D.shape)
    model_value_w=np.zeros(XM2D.shape)
    model_value_p=np.zeros(XM2D.shape)
    for t in range(len(list_t)):
        for i in range(len(list_x)):
            for j in range(len(list_y)):
                coords = torch.tensor([[list_x[i],list_y[j],z_position,list_t[t]]], dtype=torch.float32)
                model_value_u[i,j] = model(tp.spaces.Points(coords, X*Y*Z*T)).as_tensor[0,0]
                model_value_v[i,j] = model(tp.spaces.Points(coords, X*Y*Z*T)).as_tensor[0,1]
                model_value_w[i,j] = model(tp.spaces.Points(coords, X*Y*Z*T)).as_tensor[0,2]
                model_value_p[i,j] = model(tp.spaces.Points(coords, X*Y*Z*T)).as_tensor[0,3]
        plt.contourf(XM2D,YM2D,model_value_u)
        plt.colorbar()
        plt.savefig(f"./figs_turb_flat/fig_turb{t}.jpg")
        plt.close()
    #plt.plot([0.4,0.4],[0,2])


In [ ]:
produce_sample(4,2,2,10)


In [ ]:
model_values=produce_sample(2,2,100)


In [ ]:
for j in np.linspace(0,2,10):
    coords = torch.tensor([[j,i] for i in np.linspace(0,2,100)], dtype=torch.float32)
    model_out = model(tp.spaces.Points(coords, X*Y)).as_tensor
    for k in range(2,5):
        plt.plot(np.linspace(0,2,100),np.array(model_out.transpose(0,1)[k].tolist()))
    #plt.plot([0.4,0.4],[0.1,-0.1])

In [ ]:
coords = torch.tensor([[0,i] for i in np.linspace(0,2,100)], dtype=torch.float32)
model_out = model(tp.spaces.Points(coords, X*Y)).as_tensor
for k in range(2,5):
    plt.plot(np.linspace(0,2,100),np.array(model_out.transpose(0,1)[k].tolist()))
    #plt.plot([0.4,0.4],[0.1,-0.1])

In [ ]:
coords = torch.tensor([[1,i] for i in DF_Data["y"]], dtype=torch.float32)
model_out = model(tp.spaces.Points(coords, X*Y)).as_tensor
for k in range(2,5):
    plt.plot(DF_Data["y"],np.abs((np.array(model_out.transpose(0,1)[k].tolist()))))
    #plt.plot([0.4,0.4],[0.1,-0.1])
plt.plot(DF_Data["y"],DF_Data["urms"],'--')
plt.plot(DF_Data["y"],DF_Data["vrms"],'--')
plt.plot(DF_Data["y"],DF_Data["uv"],'--')

In [ ]:
coords = torch.tensor([[1,i] for i in DF_Data["y"]], dtype=torch.float32)
model_out = model(tp.spaces.Points(coords, X*Y)).as_tensor
for k in range(0,2):
    plt.plot(DF_Data["y"],np.array(model_out.transpose(0,1)[k].tolist()))
    #plt.plot([0.4,0.4],[0.1,-0.1])
plt.plot(DF_Data["y"],DF_Data["U"],'--')
plt.plot(DF_Data["y"],DF_Data["V"],'--')


In [ ]:
DF_Data

In [ ]:
for j in np.linspace(0,2,10):
    coords = torch.tensor([[j,i,20] for i in np.linspace(0,2,100)], dtype=torch.float32)
    model_out = model(tp.spaces.Points(coords, X*Y*H)).as_tensor
    plt.plot(np.linspace(0,2,100),np.array(model_out.transpose(0,1)[0].tolist()))
    plt.plot([0.3,0.3],[0.1,-0.1])

In [ ]:
coords = torch.tensor([[5,i] for i in np.linspace(0,2.0,100)], dtype=torch.float32)
model_out = model(tp.spaces.Points(coords, X*Y)).as_tensor
plt.plot(np.array(model_out.transpose(0,1)[0].tolist()))
coords = torch.tensor([[0,i] for i in np.linspace(0,2.0,100)], dtype=torch.float32)
model_out = model(tp.spaces.Points(coords, X*Y)).as_tensor
plt.plot(np.array(model_out.transpose(0,1)[0].tolist()))